In [5]:
import cv2

import sys
import os
import datetime
import threading
import queue


# Add the src directory to the sys.path list
sys.path.insert(0, os.path.abspath('../src'))
from services.event_service import add_event
from db_models.init_db import create_tables
from surveillance_daemon.object_detector import ObjectDetector
from surveillance_daemon.video_capture import VideoCapture
from surveillance_daemon.motion_detector import MotionDetector
from surveillance_daemon.video_recorder import VideoRecorder
from surveillance_daemon.notification import Notification

In [6]:
# Initialize the database session
create_tables()

# Initialize system components
camera = VideoCapture(device=0)
object_detector = ObjectDetector(
    "../models/yolov7-tiny/yolov7-tiny.cfg",
    "../models/yolov7-tiny/yolov7-tiny.weights",
    "../data/coco.names",
)
motion_detector = MotionDetector()
notification = Notification(to_email="gunes314@gmail.com")

2024-05-16 22:48:04,294 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 22:48:04,294 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("events")
2024-05-16 22:48:04,294 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-16 22:48:04,295 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
# Queue to store video paths to be processed
video_queue = queue.Queue()

def worker():
    """Worker thread that processes video files from the queue."""
    while True:
        video_path = video_queue.get()
        if video_path is None:
            break
        detected_objects, video_path = object_detector.process_video(video_path)
        add_event(video_path, str(detected_objects))
        # send a notification if an object is detected
        if detected_objects:
            notification.send(detected_objects)
        video_queue.task_done()

# Create and start the worker thread
num_worker_threads = 1
threads = []
for i in range(num_worker_threads):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)

In [8]:
recording = False
start_time = None

while True:
    frame = camera.read_frame()
    if frame is None:
        break

    font = cv2.FONT_HERSHEY_SIMPLEX
    color = (0, 255, 0)

    # Detect motion and start recording video
    if motion_detector.detect_motion(frame):
        cv2.putText(frame, "M", (10, 30), font, 0.5, color, 2)
        if not recording:
            recording = True
            start_time = datetime.datetime.now()
            video_recorder = VideoRecorder(folder="../videos", fps=camera.get_fps(), width=camera.get_frame_width(), height=camera.get_frame_height())
    
    # Record video for 5 seconds
    if recording:
        cv2.putText(frame, "R", (30, 30), font, 0.5, color, 2)
        video_recorder.write_frame(frame)
        if (datetime.datetime.now() - start_time).total_seconds() > 5:
            # Start object detection on a separate thread when video recording stops
            video_queue.put(video_recorder.get_video_path())

            # Clear the video recorder object
            recording = False
            video_recorder.release()
            del video_recorder

    cv2.imshow("Object detection", frame)
    if cv2.waitKey(1) == 27:
        break

camera.release()
cv2.destroyAllWindows()

2024-05-16 22:48:14,758 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 22:48:14,759 INFO sqlalchemy.engine.Engine INSERT INTO events (video_path, objects, created_at) VALUES (?, ?, ?)
2024-05-16 22:48:14,759 INFO sqlalchemy.engine.Engine [cached since 102.5s ago] ('../videos/2024-05-16T22:48:07_annotated.avi', "{'sofa', 'bed', 'tvmonitor'}", datetime.datetime(2024, 5, 16, 20, 46, 21, 31408, tzinfo=datetime.timezone.utc))
2024-05-16 22:48:14,759 INFO sqlalchemy.engine.Engine COMMIT
{'sofa', 'bed', 'tvmonitor'}
2024-05-16 22:48:25,219 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-16 22:48:25,220 INFO sqlalchemy.engine.Engine INSERT INTO events (video_path, objects, created_at) VALUES (?, ?, ?)
2024-05-16 22:48:25,220 INFO sqlalchemy.engine.Engine [cached since 113s ago] ('../videos/2024-05-16T22:48:18_annotated.avi', "{'bottle', 'bed', 'sofa', 'person', 'chair', 'tvmonitor'}", datetime.datetime(2024, 5, 16, 20, 46, 21, 31408, tzinfo=datetime.timezone.utc))
2024-05-16 

In [ ]:
object_detector.process_video("../videos/2024-05-15T23:14:42.avi")

In [10]:
camera.release()
cv2.destroyAllWindows()